# Ch10. 워드 임베딩 (Word Embedding)

# v04. 글로브 (GloVe)

- 글로브(Global Vectors for Word Representation, GloVe)는 카운트 기반과 예측 기반을 모두 사용하는 방법론이다.
- 2014년에 미국 스탠포드대학에서 개발한 단어 임베딩 방법론이다.
- 앞서 학습하였던 기존의 카운트 기반의 LSA(Latent Semantic Analysis)와 예측 기반의 Word2Vec의 단점을 지적하며 이를 보완한다는 목적으로 나왔다.
- 실제로도 Word2Vec 만큼 뛰어난 성능을 보여준다.
- 현재까지의 연구에 따르면 단정적으로 Word2Vec와 GloVe 중에서 어떤 것이 더 뛰어나다고 말할 수는 없고, 이 두 가지 전부를 사용해보고 성능이 더 좋은 것을 사용하는 것이 바람직하다.

<br>

## 4.1 기존 방법론에 대한 비판

- 기존의 방법론을 복습해보자.

<br>

### 4.1.1 LSA

- LSA는 DTM이나 TF-IDF 행렬과 같이 각 문서에서의 각 단어의 빈도 수를 카운트한 행렬이라는 전체적인 통계 정보를 입력으로 받아 차원을 축소(Truncated SVD)하여 잠재된 의미를 끌어내는 방법론이다.

<br>

### 4.1.2 Word2Vec

- 반면, Word2Vec는 실제값과 예측값에 대한 오차를 손실 함수를 통해 줄여나가며 학습하는 예측 기반의 방법론이다.

<br>

### 4.1.3 각각의 방법론의 장단점

- 서로 다른 방법을 사용하는 이 두 방법론은 각각 장, 단점이 있다.

- LSA
  - (장점) : 카운트 기반으로 코퍼스의 전체적인 통계 정보를 고려한다.
  - (단점) : 왕 : 남자 = 여왕 : ?(정답은 여자)와 같은 단어 의미의 유추 작업(Analogy task)에는 성능이 떨어진다.

- Word2Vec
  - (장점) : 예측 기반으로 단어 간 유추 작업에는 LSA보다 뛰어나다.
  - (단점) : 임베딩 벡터가 윈도우 크기 내에서만 주변 단어를 고려하기 때문에 코퍼스의 전체적인 통계 정보를 반영하지 못한다.

<br>

### 4.1.4 GloVe

- GloVe는 이러한 기존 방법론들의 각각의 한계를 지적하며, LSA의 메커니즘이었던 카운트 기반의 방법과 Word2Vec의 메커니즘이었던 예측 기반의 방법론 두 가지를 모두 사용한다.

<br>

## 4.2 윈도우 기반 동시 등장 행렬 (Window based Co-occurrence Matrix)

### 4.2.1 단어의 동시 등장 행렬

- 행과 열을 전체 단어 집합의 단어들로 구성
- i 단어의 윈도우 크기(Window Size) 내에서 k 단어가 등장한 횟수를 i행 k열에 기재한 행렬

<br>

### 4.2.2 예제

- 아래와 같은 텍스트가 있다고 하자.

> I like deep learning  
I like NLP  
I enjoy flying

- 윈도우 크기가 N 일 때는 좌, 우에 존재하는 N개의 단어만 참고하게 된다.
- 윈도우 크기가 1일 때, 위의 텍스트를 가지고 구성한 동시 등장 행렬은 다음과 같다.

| 카운트   | I    | like | enjoy | deep | learning | NLP  | flying |
| :------- | :--- | :--- | :---- | :--- | :------- | :--- | :----- |
| I        | 0    | 2    | 1     | 0    | 0        | 0    | 0      |
| like     | 2    | 0    | 0     | 1    | 0        | 1    | 0      |
| enjoy    | 1    | 0    | 0     | 0    | 0        | 0    | 1      |
| deep     | 0    | 1    | 0     | 0    | 1        | 0    | 0      |
| learning | 0    | 0    | 0     | 1    | 0        | 0    | 0      |
| NLP      | 0    | 1    | 0     | 0    | 0        | 0    | 0      |
| flying   | 0    | 0    | 1     | 0    | 0        | 0    | 0      |

- 해당 테이블은 스탠포드 대학교의 자연어 처리 강의를 참고하였다. ([링크](http://web.stanford.edu/class/cs224n/slides/cs224n-2019-lecture02-wordvecs2.pdf))

<br>

### 4.2.3 윈도우 기반 동시 등장 행렬의 특징

- 위 행렬은 행렬을 전치(Transpose)해도 동일한 행렬이 된다는 특징이 있다.
- 그 이유는 i 단어의 윈도우 크기 내에서 k 단어가 등장한 빈도는 반대로 k 단어의 윈도우 크기 내에서 i 단어가 등장한 빈도와 동일하기 때문이다.

<br>

## 4.3 동시 등장 확률 (Co-occurrence Probability)

- 아래의 표는 어떤 동시 등장 행렬을 가지고 정리한 동시 등장 확률(Co-occurrence Probability)을 보여준다.
- 그렇다면 동시 등장 확률이란 무엇일까?

<br>

### 4.3.1 동시 등장 확률 $P(k \, | \, i)$

- 동시 등장 행렬로부터 특정 단어 i의 전체 등장 횟수를 카운트한다.
- 특정 단어 i가 등장했을 때 어떤 단어 k가 등장한 횟수를 카운트하여 계산한 조건부 확률이다.

- $P(k \, | \, i)$에서 i를 중삼 단어(Center Word), k를 주변 단어(Context Word)라고 할 때
  - 분모값 : 동시 등장 행렬에서 중심 단어 i의 행의 모든 값을 더한 값
  - 분자값 : i행 k열의 값

$$
P(k \, | \, i) = \frac{\text{value of the i row and k column}}{\text{sum of all values in i row}}
$$

<br>

### 4.3.2 표로 정리한 동시 등장 확률

- 다음은 GloVe의 제안 논문에서 가져온 동시 등장 확률을 표로 정리한 하나의 예이다.

| 동시 등장 확률과 크기 관계 비(ratio) | k=solid  | k=gas    | k=water | k=fasion |
| :----------------------------------- | :------- | :------- | :------ | :------- |
| P(k l ice)                           | 0.00019  | 0.000066 | 0.003   | 0.000017 |
| P(k l steam)                         | 0.000022 | 0.00078  | 0.0022  | 0.000018 |
| P(k l ice) / P(k l steam)            | 8.9      | 0.085    | 1.36    | 0.96     |

<br>

#### 4.3.2.1 k = solid

- `solid`가 등장했을 때 `ice`가 등장할 확률인 0.00019은 `solid`가 등장했을 때 `steam`이 등장할 확률인 0.000022보다 약 8.9배 더 크다.
- `solid`는 '단단한'이라는 의미를 가졌으므로, '증기'라는 의미를 가지는 `steam`보다는 당연히 '얼음'의 뜻을 가진 `ice`라는 단어와 더 자주 등장할 것이다.

- k가 `solid`일 때, $P(\text{solid} \, | \, \text{ice}) / P(\text{solid} \, | \, \text{steam})$를 계산한 값은 8.9가 나온다.
- 이 값은 1보다는 매우 큰 값이다. ($P(\text{solid} \, | \, \text{ice})$의 값은 크고, $P(\text{solid} \, | \, \text{steam})$의 값은 작기 때문)

<br>

#### 4.3.2.2 k = gas

- `gas`는 `ice`보다는 `steam`과 더 자주 등장한다.
- 그러므로 $P(\text{gas} \, | \, \text{ice}) / P(\text{gas} \, | \, \text{steam})$를 계산한 값은 0.085가 나온다.
- 이 값은 1보다 훨씬 작은 값이다.

<br>

#### 4.3.2.3 k = water

- k가 `water`인 경우에는 `solid`와 `steam` 두 단어 모두와 동시 등장하는 경우가 많으므로 1에 가까운 값(1.36)이 나온다.

<br>

#### 4.3.2.4 k = fasion

- k가 `fasion`인 경우에는 `solid`와 `steam` 두 단어 모두와 동시 등장하는 경우가 적으므로 1에 가까운 값(0.96)에 가까운 값이 나온다.

<br>

### 4.3.3 표 내용 정리

- 보기 쉽도록 조금 단순화해서 표현한 표는 다음과 같다.

| 동시 등장 확률과 크기 관계 비(ratio) | k=solid | k=gas   | k=water    | k=fasion   |
| :----------------------------------- | :------ | :------ | :--------- | :--------- |
| P(k l ice)                           | 큰 값   | 작은 값 | 큰 값      | 작은 값    |
| P(k l steam)                         | 작은 값 | 큰 값   | 큰 값      | 작은 값    |
| P(k l ice) / P(k l steam)            | 큰 값   | 작은 값 | 1에 가까움 | 1에 가까움 |

<br>

## 4.4 손실 함수 (Loss function)

### 4.4.1 손실 함수 관련 용어 정리

- $X$
  - 동시 등장 행렬 (Co-occurrence Matrix)

- $X_{ij}$
  - 중심 단어 $i$가 등장했을 때 윈도우 내 주변 단어 $j$가 등장하는 횟수

- $X_i$ ($= \sum_j X_{ij}$)
  - 동시 등장 행렬에서 $i$행의 값을 모두 더한 값

- $P_{ik}$ ($= P(k \, | \, i) \; = \; \frac{X_{ik}}{X_{i}}$)
  - 중심 단어 $i$가 등장했을 때, 윈도우 내 주변 단어 $k$가 등장할 확률
  - ex) $P(\text{solid} \, | \, \text{ice})$ = 단어 `ice`가 등장했을 때, 단어 `solid`가 등장할 확률

- $\frac{P_{ik}}{P_{jk}}$
  - $P_{ik}$를 $P_{jk}$로 나눠준 값
  - ex) $P(\text{solid} \, | \, \text{ice}) / P(\text{solid} \, | \, \text{steam}) = 8.9$

- $w_i$
  - 중심 단어 $i$의 임베딩 벡터

- $\tilde{w_k}$
  - 주변 단어 $k$의 임베딩 벡터

<br>

### 4.4.2 GloVe의 아이디어

> "임베딩된 중심 단어와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 확률이 되도록 만드는 것"

- 즉, 이를 만족하도록 임베딩 벡터를 만드는 것이 목표이다.

- 이를 식으로 표현하면 다음과 같다.

$
\qquad
dot \; product \left( w_i \; \tilde{w_k} \right) \; \approx \; P(k \, | \, i) = P_{ik}
$

- 더 정확히는 GloVe는 아래와 같은 관계를 가지도록 임베딩 벡터를 설계한다.

$
\qquad
dot \; product \left( w_i \; \tilde{w_k} \right) \; \approx \; log \; P(k \, | \, i) = log \; P_{ik}
$

<br>

### 4.4.3 임베딩 벡터들을 만들기 위한 손실 함수 설계

- 가장 중요한 것은 **단어 간의 관계를 잘 표현하는 함수여야 한다는 것**이다.
- 이를 위해 앞서 배운 개념인 $P_{ik} / P_{jk}$를 식에 사용한다.

<br>

#### 4.4.3.1 초기 식

- GloVe의 연구진들은 벡터 $w_i$, $w_j$, $\tilde{w_k}$를 가지고 어떤 함수 $F$를 수행하면, $P_{ik} / P_{jk}$가 나온다는 초기 식으로부터 전개를 시작한다.

$
\qquad
F(w_i, \; w_j, \; \tilde{w_k}) = \frac{P_{ik}}{P_{jk}}
$

- 아직 이 함수 $F$가 어떤 식을 가지고 있는 지는 정해진 게 없다.
- 위의 목적에 맞게 근사할 수 있는 함수식은 무수히 많을 것이다.
- 그 중 최적의 식에 다가가기 위해서 차근 차근 디테일을 추가해보자.

<br>

#### 4.4.3.2 뺄셈 적용

- 함수 $F$는 두 단어 사이의 동시 등장 확률의 크기 관계 비(ratio) 정보를 벡터 공간에 인코딩하는 것이 목적이다.
- 이를 위해 GloVe 연구진들은 $w_i$와 $w_j$라는 두 벡터의 차이를 함수 $F$의 입력으로 사용하는 것을 제안한다.

$
\qquad
F(w_i - w_j, \; \tilde{w_k}) = \frac{P_{ik}}{P_{jk}}
$

<br>

#### 4.4.3.3 내적 적용

- 그런데 위 식에서 우변($\frac{P_{ik}}{P_{jk}}$)은 스칼라 값이고, 좌변은 벡터값이다.
- 이를 성립하게 해주기 위해서 함수 $F$의 두 입력에 내적(Dot Product)을 수행한다.

$
\qquad
F( \left( w_i - w_j \right)^T \cdot \tilde{w_k}) = \frac{P_{ik}}{P_{jk}}
$

<br>

#### 4.4.3.4 정리

- 정리하면, 선형 공간(Linear Space)에서 단어의 의미 관계를 표현하기 위해 **뺄셈**과 **내적**을 택했다.

<br>

#### 4.4.3.5 준동형(Homomorphism)

- 여기서 함수 $F$가 만족해야 할 필수 조건이 있다.
- 중심 단어 $w$와 주변 단어 $\tilde{w}$ 라는 선택 기준은 실제로는 **무작위 선택**이다.
- 그러므로 이 둘의 관계는 **자유롭게 교환될 수 있도록** 해야 한다.
- 이 것이 성립되게 하기 위해서 GloVe 연구진은 함수 $F$가 **실수의 덧셈**과 **양수의 곱셈**에 대해서 **준동형(Homomorphism)**을 만족하도록 한다.
- 정리하면 $a$와 $b$에 대해서 함수 $F$가 $F(a+b)$가 $F(a)F(b)$와 같도록 만족시켜야 한다는 의미이다.

- 식으로 나타내면 아래와 같다.

$
\qquad
F(a + b) = F(a) F(b) \quad \forall a, \, b \; \in \; \mathbb{R}
$

<br>

#### 4.4.3.6 준동형식 변형 (1)

- 이제 이 준동형식을 현재 전개하던 GloVe 식에 적용할 수 있도록 조금씩 바꿔보자.
- 전개하던 GloVe 식에 따르면, 함수 $F$는 결과값으로 스칼라 값인 $\frac{P_{ik}}{P_{jk}}$이 나와야 한다.
- 준동형식에서  
  - $a$와 $b$가 각각 벡터값인 경우 $\rightarrow$ 함수 $F$의 결과값으로는 스칼라 값이 나올 수 없다.
  - $a$와 $b$가 각각 **두 벡터의 내적값**인 경우 $\rightarrow$ 함수 $F$의 결과값으로 스칼라 값이 나올 수 있다.

- 그러므로 위의 준동형식을 아래와 같이 바꿔보자.
- 여기서 $v_1, \, v_2, \, v_3, \, v_4$는 각각 벡터값이다.
- 아래의 $V$는 벡터를 의미한다.

$
\qquad
F \left( v_1^T \, v_2 + v_3^T \, v_4 \right) = F(v_1^T \, v_2) \, F(v_3^T \, v_4) \quad \forall \, v_1, \, v_2, \, v_3, \, v_4 \; \in \; V
$

<br>

#### 4.4.3.7 준동형식 변형 (2)

- 그런데 앞서 작성한 GloVe 식에서는 $w_i$와 $w_j$라는 두 벡터의 차이를 함수 $F$의 입력으로 받았다.
- GloVe 식에 바로 적용을 위해 준동형식을 뺄셈에 대한 준동형식으로 변경한다.
- 그렇게 되면 곱셈도 나눗셈으로 바뀐다.

$
\qquad
F \left( v_1^T \, v_2 - v_3^T \, v_4 \right) = \frac{F(v_1^T \, v_2)}{F(v_3^T \, v_4)} \quad \forall \, v_1, \, v_2, \, v_3, \, v_4 \; \in \; V
$

<br>

#### 4.4.3.8 준동형식 적용

- 이제 이 준동형식을 GloVe 식에 적용해보자.
- 우리의 목적은 함수 F의 우변을 다음과 같이 바꿔야 한다.

$
\qquad
F( \left( w_i - w_j \right)^T \cdot \tilde{w_k}) = \frac{F(w_i^T \, \tilde{w_k})}{F(w_j^T \, \tilde{w_k})}
$

- 이전 식에 따르면 우변은 본래 $\frac{P_{ik}}{P_{jk}}$ 이였으므로, 결과적으로 다음과 같다.

$
\qquad
\frac{P_{ik}}{P_{jk}} = \frac{F(w_i^T \, \tilde{w_k})}{F(w_j^T \, \tilde{w_k})}
$

$
\qquad
F(w_i^T \, \tilde{w_k}) = P_{ik} = \frac{X_{ik}}{X_i}
$

- 좌변을 풀어쓰면 다음과 같다.

$
\qquad
F( w_i^T \, \tilde{w_k} \; - \; w_j^T \, \tilde{w_k}) =
\frac{F(w_i^T \, \tilde{w_k})}{F(w_j^T \, \tilde{w_k})}
$

- 이는 뺄셈에 대한 준동형식의 형태와 정확히 일치한다.

<br>

#### 4.4.3.9 함수 $F$

- 이제 이를 만족하는 함수 $F$를 찾아야 한다.
- 그리고 이를 정확하게 만족시키는 함수가 있는데 바로 **지수 함수(Exponential function)**이다.
- $F$를 지수 함수 $exp$라고 해보자.

$
\qquad
exp( w_i^T \, \tilde{w_k} \; - \; w_j^T \, \tilde{w_k}) =
\frac{exp(w_i^T \, \tilde{w_k})}{exp(w_j^T \, \tilde{w_k})}
$

$
\qquad
exp(w_i^T \, \tilde{w_k}) = P_{ik} = \frac{X_{ik}}{X_i}
$

- 위의 두 번째 식으로부터 다음과 같은 식을 얻을 수 있다.

$
\qquad
w_i^T \, \tilde{w_k} = log \, P_{ik} = log \left( \frac{X_{ik}}{X_i} \right) = log \, X_{ik} - log \, X_i
$

<br>

#### 4.4.3.10 편향 적용

- 그런데 여기서 상기해야 할 것은 앞서 언급했듯이, 사실 $w_i$와 $\tilde{w}$는 두 값의 위치를 서로 바꾸어도 식이 성립해야 한다.
- $X_{ik}$의 정의를 생각해보면 $X_{ki}$와도 같다.
- 그런데 이게 성립되려면 위의 식에서 $log \, X_i$ 항이 걸림돌이다.
- 이 부분만 없다면 이를 성립시킬 수 있다.
- 그래서 GloVe 연구팀은 $log \, X_i$ 항을 $w_i$에 대한 편향 $b_i$라는 상수항으로 대체하기로 한다.
- 같은 이유로 $\tilde{w_k}$에 대한 편향 $\tilde{b_k}$를 추가한다.

$
\qquad
w_i^T \, \tilde{w_k} + b_i + \tilde{b_k} = log \, X_{ik}
$

<br>

#### 4.4.3.11 손실 함수 일반화

- 위의 식이 손실 함수의 핵심이 되는 식이다.
- 우변의 값과의 차이를 최소화하는 방향으로 좌변의 4개의 항은 학습을 통해 값이 바뀌는 변수들이 된다.
- 즉, 손실 함수는 다음과 같이 일반화될 수 있다.

$
\qquad
Loss \, function = \sum_{m,n=1}^V \left( w_m^T \, \tilde{w_n} + b_m + \tilde{b_n} - log \, X_{mn} \right)^2
$

- 여기서 $V$는 단어 집합의 크기를 의미한다.

<br>

#### 4.4.3.12 손실 함수의 보완 필요성

- 그런데 아직 최적의 손실 함수라기에는 부족하다.
- GloVe 연구진은 $log \, X_{ik}$에서 $X_{ik}$ 값이 0이 될 수 있음을 지적한다.
- 대안 중 하나는 $log \, X_{ik}$ 항을 $log \left( 1 + X_{ik} \right)$로 변경하는 것이다.
- 하지만 이렇게 해도 여전히 해결되지 않는 문제가 있다.
- 바로 **동시 등장 행렬 $X$는 마치 DTM처럼 희소 행렬(Sparse Matrix)일 가능성이 다분하다는 점**이다.

<br>

#### 4.4.3.13 가중치 함수 도입

- 동시 등장 행렬 $X$에는 많은 값이 0 이거나, 동시 등장 빈도가 적어서 많은 값이 작은 수치를 가지는 경우가 많다.
- 앞서 빈도수를 가지고 가중치를 주는 고민을 하는 TF-IDF나 LSA와 같은 몇 가지 방법들을 본 적이 있다.
- GloVe의 연구진은 동시 등장 행렬에서 동시 등장 빈도의 값 $X_{ik}$이 굉장히 낮은 경우에는 정보에 거의 도움이 되지 않는다고 판단한다.
- 그래서 이에 대한 가중치를 주는 고민을 하게 된다.
- GloVe 연구팀이 선택한 것은 바로 **$X_{ik}$의 값에 영향을 받는 가중치 함수(Weighting function) $f \left( X_{ik} \right)$를 손실 함수에 도입하는 것**이다.

- GloVe에 도입되는 $f \left( X_{ik} \right)$의 그래프는 아래와 같다.

$\quad$ ![](https://wikidocs.net/images/page/22885/%EA%B0%80%EC%A4%91%EC%B9%98.PNG)

- $X_{ik}$의 값이 작다 $\rightarrow$ 상대적으로 함수의 값은 작도록 한다.
- $X_{ik}$의 값이 크다 $\rightarrow$ 상대적으로 함수의 값은 크도록 한다.

- 하지만 $X_{ik}$가 지나치게 높다고 해서 지나친 가중치를 주지 않기 위해서 또한 함수의 최대값이 정해져 있다. (최대값은 1)
  - ex) 'It is'와 같은 불용어의 동시 등장 빈도수가 높다고 해서 지나친 가중을 받아서는 안된다.

- 이 함수의 값을 손실 함수에 곱해주면 가중치의 역할을 할 수 있다.
- 이 함수 $f(x)$의 식은 다음과 같이 정의된다.

$
\qquad
f(x) = min \left( 1, \; (x / x_{max})^{3/4} \right)
$

<br>

#### 4.4.3.14 최종 일반화된 손실 함수

- 최종적으로 다음과 같은 일반화된 손실 함수를 얻어낼 수 있다.

$
\qquad
Loss \, function = \sum_{m,n=1}^V f \left( X_{mn} \right) \, \left( w_m^T \, \tilde{w_n} + b_m + \tilde{b_n} - log \, X_{mn} \right)^2
$

<br>

## 4.5 GloVe 훈련시키기